![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Disponibilización de modelos

En este notebook aprenderá a guardar un modelo y a disponibilizarlo como una API con la librería Flask. Una API (interfaz de programación de aplicaciones) es un conjunto de definiciones y protocolos que permiten que servicios, en este caso modelos, retornen resultados y respuestas sin necesidad de saber cómo están implementados.

## Instrucciones Generales:

Este notebook esta compuesto por dos secciones. En la primera sección, se entrena y guarda (exportar) un modelo de XGBoost con Calibración para predecir el precio de un automóvil. En la segunda parte, se usa el modelo entrenado y se disponibiliza usando la libreria *Flask*. 


## Importar base de datos y librerías

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías

import numpy as np
import pandas as pd
import seaborn as sns


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import randint, uniform

from xgboost import XGBRegressor

import joblib
#import os
#os.chdir('..')

In [3]:
# Carga de datos de archivos .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/rofegobu/ML_LPN_RGB/main/dataTrain_carListings.csv')


## Identificar y Eliminar Valores Atípicos (Outliers) - Filtrado 1 y 2

In [4]:
## Identificar y Eliminar Valores Atípicos (Outliers) - Filtrado 1

# Identificar y eliminar valores atípicos de 'Price' y 'Milage' para cada 'Model' y 'Year',

# Agrupar por 'Year' y 'Model' y calcular la media y la desviación estándar para 'Price' y 'Mileage'
grouped = dataTraining.groupby(['Year', 'Model']).agg({
    'Price': ['mean', 'std'],
    'Mileage': ['mean', 'std']
}).reset_index()

# Aplanar encabezado de las columnas multi-índice
grouped.columns = ['Year', 'Model', 'Price_mean', 'Price_std', 'Mileage_mean', 'Mileage_std']

# Unir los cálculos anteriores al DataFrame original para tener los valores de referencia por año y modelo
dataTraining = dataTraining.merge(grouped, on=['Year', 'Model'], how='left')

# Identificar valores atípicos usando la media y la desviación estándar por año y modelo
outliers = dataTraining[
    ((dataTraining['Price'] < (dataTraining['Price_mean'] - 1.5 * dataTraining['Price_std'])) |
     (dataTraining['Price'] > (dataTraining['Price_mean'] + 1.5 * dataTraining['Price_std']))) |
    ((dataTraining['Mileage'] < (dataTraining['Mileage_mean'] - 1.5 * dataTraining['Mileage_std'])) |
     (dataTraining['Mileage'] > (dataTraining['Mileage_mean'] + 1.5 * dataTraining['Mileage_std'])))
]

# Eliminar filas con valores atípicos
dataTraining_filtrado1 = dataTraining.drop(outliers.index)

# Eliminar columnas auxiliares usadas para los cálculos
dataTraining_filtrado1 = dataTraining_filtrado1.drop(columns=['Price_mean', 'Price_std', 'Mileage_mean', 'Mileage_std'])

# Mostrar DataFrame actualizado
#print(dataTraining_filtrado1)


## Identificar y Eliminar Valores Atípicos (Outliers) - Filtrado 2

# Identificar y eliminar valores atípicos de 'Price' para cada 'Model', 'Year' y 'Mileage_range'

# Crear la columna 'Mileage_range'
dataTraining_filtrado1['Mileage_range'] = (np.ceil(dataTraining_filtrado1['Mileage'] / 5000) * 5000).astype(int)

# Agrupar por 'Year', 'Model', y 'Mileage_range' y calcular la media y la desviación estándar para 'Price'
grouped = dataTraining_filtrado1.groupby(['Year', 'Model', 'Mileage_range']).agg({
    'Price': ['mean', 'std']
}).reset_index()

# Aplanar encabezado de las columnas multi-índice
grouped.columns = ['Year', 'Model', 'Mileage_range', 'Price_mean', 'Price_std']

# Unir los cálculos anteriores al DataFrame original para tener los valores de referencia
dataTraining_filtrado1 = dataTraining_filtrado1.merge(grouped, on=['Year', 'Model', 'Mileage_range'], how='left')

# Identificar los valores atípicos usando la media y la desviación estándar para 'Price'
outliers = dataTraining_filtrado1[
    ((dataTraining_filtrado1['Price'] < (dataTraining_filtrado1['Price_mean'] - 1.5 * dataTraining_filtrado1['Price_std'])) |
     (dataTraining_filtrado1['Price'] > (dataTraining_filtrado1['Price_mean'] + 1.5 * dataTraining_filtrado1['Price_std'])))
]

# Eliminar filas con valores atípicos
dataTraining_filtrado2 = dataTraining_filtrado1.drop(outliers.index)

# Eliminar columnas auxiliares usadas para los cálculos
dataTraining_filtrado2 = dataTraining_filtrado2.drop(columns=['Price_mean', 'Price_std', 'Mileage_range'])

# Mostrar DataFrame actualizado
#print(dataTraining_filtrado2)


In [5]:
# Extraer valores únicos de las columnas State Make y Model
states = dataTraining['State'].unique().tolist()
makes = dataTraining['Make'].unique().tolist()
models = dataTraining['Model'].unique().tolist()

# Escribir los valores únicos a un archivo .py dentro del directorio 'content'
with open('content/unique_values.py', 'w') as file:
    file.write("STATES = " + str(states) + "\n")
    file.write("MAKES = " + str(makes) + "\n")
    file.write("MODELS = " + str(models) + "\n")

## Transformación de características numéricas y categóricas - Pipeline de preprocesamiento - División de los datos en conjuntos de entrenamiento y validación 

In [6]:
## Transformación de características numéricas y categóricas - Pipeline de preprocesamiento - División de los datos en conjuntos de entrenamiento y validación 


# Selección de características
numeric_features = ['Year', 'Mileage']
categorical_features = ['State', 'Make', 'Model']

# Pipeline para características numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

# Pipeline para características categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combinar transformadores en un preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Separación de variables predictoras (X) y variable de interés (y)
y = dataTraining_filtrado2['Price']
X = dataTraining_filtrado2.drop('Price', axis=1)

# División en conjuntos de entrenamiento y validación
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

# Aplicar el preprocesador
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_valid_preprocessed = preprocessor.transform(X_valid)


In [7]:
# Exportar el preprocesador entrenado
joblib.dump(preprocessor, 'content/preprocessor.pkl')

['content/preprocessor.pkl']

## Entrenar y guardar modelo XGBoost

In [8]:
## Entrenar y guardar modelo XGBoost

# Configurar modelo XGBoost con los mejores parámetros encontrados
xgb_model = XGBRegressor(
    colsample_bytree=0.925,
    gamma=0.936,
    learning_rate=0.246,
    max_depth=10,
    n_estimators=439,
    subsample=0.790,
    random_state=42
)

# Entrenar modelo con datos de entrenamiento preprocesados
xgb_model.fit(X_train_preprocessed, y_train)

# Predicir en el conjunto de validación
y_pred = xgb_model.predict(X_valid_preprocessed)

# Calcular RMSE para el conjunto de validación
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print(f"RMSE XGBoost post-calibración en conjunto de validación: {rmse:.2f}")


RMSE XGBoost post-calibración en conjunto de validación: 2225.45


In [9]:
# Exportar modelo de regresión a archivo binario .pkl
joblib.dump(xgb_model, 'content/price_predictor_xgb.pkl', compress=3)

['content/price_predictor_xgb.pkl']

## Disponibilizar modelo con Flask

In [10]:
## Disponibilizar modelo con Flask

from flask import Flask, request, jsonify
from flask_restx import Api, Resource, fields
import joblib
import numpy as np
import pandas as pd

from content.unique_values import STATES, MAKES, MODELS

app = Flask(__name__)

api = Api(
    app, 
    version='1.0', 
    title='API de Predicción de Precios - Automóviles usados',
    description='API para predecir precios de automóviles usados en EE.UU.')


# Espacio de nombres para la API
ns = api.namespace('predict', description='Predicciones')


# Modelo de datos de entrada de la API
model = api.model('PredictionInput', {
    'Year': fields.Integer(required=True, description='Año del vehículo', example=2016),
    'Mileage': fields.Float(required=True, description='Kilometraje del vehículo', example=50000),
    'State': fields.String(required=True, description='Estado donde se encuentra el vehículo', enum=STATES),
    'Make': fields.String(required=True, description='Marca del vehículo', enum=MAKES),
    'Model': fields.String(required=True, description='Modelo del vehículo', enum=MODELS)
})

# Cargar el modelo y preprocesador entrenados
model_auto_price = joblib.load('content/price_predictor_xgb.pkl')
preprocessor = joblib.load('content/preprocessor.pkl')

# Ruta del recurso de predicción   
@ns.route('/')
class Predict(Resource):
    @api.expect(model)
    def post(self):
        data = request.json
        input_df = pd.DataFrame([data])
        preprocessed_data = preprocessor.transform(input_df)
        prediction = model_auto_price.predict(preprocessed_data)
        # Convert numpy float32 to Python float
        prediction = float(prediction[0])
        return jsonify({'predicted_price': prediction})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
Traceback (most recent call last):
  File "/Users/robertogb/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 15, in <module>
    from ipykernel import kernelapp as app
  File "/Users/robertogb/anaconda3/lib/python3.11/site-packages/ipykernel/__init__.py", line 5, in <module>
    from .connect import *  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/robertogb/anaconda3/lib/python3.11/site-packages/ipykernel/connect.py", line 11, in <module>
    import jupyter_client
  File "/Users/robertogb/anaconda3/lib/python3.11/site-packages/jupyter_client/__init__.py", line 8, in <module>
    from .asynchronous import AsyncKernelClient  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/robertogb/anaconda3/lib/python3.11/site-packages/jupyter_client/asynchronous/__init__.py", line 1, in <module>
    from .client import AsyncKernelClient  # noqa
    ^^^^^^^^^^^^^^^^

SystemExit: 1

In [ ]:
# Ejecución de la aplicación que disponibiliza el modelo de manera local en el puerto 5000
app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.1.100:5001
Press CTRL+C to quit
127.0.0.1 - - [27/Apr/2024 19:42:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2024 19:42:02] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [27/Apr/2024 19:42:02] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
127.0.0.1 - - [27/Apr/2024 19:42:02] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Apr/2024 19:42:02] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Apr/2024 19:42:03] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2024 20:10:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2024 20:10:30] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
127.0.0.1 - - [27/Apr/2024 20:10:30] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [27/Apr/2024 20:10:30] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Apr/2024 20:10:30] "GET /sw

El modelo debe haber quedado disponibilizado en el puerto 5000. Para predecir la probabilidad de que una URL sea fraudulenta (phishing) copie en la barra de busqueda de su navegador la siguiente dirección (http://localhost:5000/predict/?URL=) y agregregue al final de esta la URL que desee precir. Por ejemplo, al copiar la URL http://localhost:5000/predict/?URL=http://consultoriojuridico.co/pp/www.paypal.com/, la API retornará la probabilidad de que la URL http://consultoriojuridico.co/pp/www.paypal.com/ sea phishing.